# Variational Inference

2024-03-19


## Example 1: Gaussian-Gamma (Conjugate) posterior

### 1. Variational inference

对于某一元高斯分布，$\mathcal N(x | \mu, \tau^{-1})$ (这里用精度 $\tau = 1 / \sigma^2$ 来代替方差表示方法)，假设我们有 $N$ 个观测数据 $X = \{x_1, x_2, \dots, x_N\}$，那么似然函数为：

**likelihood：**
$$\begin{aligned}
    p(X | \mu, \tau) &= \prod_{i=1}^N (\frac{\tau}{2 \pi})^{1/2} \exp\bigg\{-\frac{\tau}{2} (x_i - \mu)^2 \bigg\} \\
     &= (\frac{\tau}{2 \pi})^{N / 2} \exp\bigg\{- \frac{\tau}{2} \sum_{i=1}^N (x_i - \mu)^2\bigg\}
\end{aligned}$$
它的共轭先验分布为 **Gaussian-Gamma分布**：

**prior：**
$$
    p(\mu | \tau) = \mathcal N(\mu_0, (\lambda_0 \tau)^{-1}) \propto \exp\big\{\frac{-\lambda_0 \tau}{2} (\mu - \mu_0)^2 \big\} \\
    p(\tau) = \text{Gamma}(\tau | a_0, b_0) \propto \tau^{a_0 -1} \exp\big\{-b_0 \tau \big\}
$$ 
可以看到这里 $\mu$ 和 $\tau$ 不独立。

**posterior：**
$$\begin{aligned}
    p(\mu, \tau | X) &= \frac{p(X | \mu, \tau) \, p(\mu | \tau) \, p(\tau)}{p(X)} \\
    & \propto p(X | \mu, \tau) \, p(\mu | \tau) \, p(\tau) \\
    &= \mathcal N(\mu_N, (\lambda_N \tau)^{-1})\ \text{Gamma} (\tau | a_N, b_N)
\end{aligned}$$
可以看到，一共有4个参数：$\mu_N, \lambda_N, a_N, b_N$

注意，这里是有后验的解析解的，还是从上面的式子中推导：
$$\begin{aligned}
    p(\mu, \tau | X) &\propto p(X | \mu, \tau) \, p(\mu | \tau) \, p(\tau) \\
    &\propto (\frac{\tau}{2 \pi})^{N/2} \exp\bigg\{-\frac{\tau}{2} \sum_{i=1}^N (x_i - \mu)^2 \bigg\} \, (\frac{\lambda_0 \tau}{2 \pi})^{1/2} \exp\big\{-\frac{\lambda_0 \tau}{2} (\mu - \mu_0)^2 \big\} \, \tau^{a_0 - 1} \exp\{ -b_0 \tau\} \\
    &\propto \tau^{(N+1)/2 + a_0 -1} \exp\{-b_0 \tau \} \, \exp\bigg\{-\frac{\tau}{2} \sum_{i=1}^N (x_i - \mu)^2 - \frac{\lambda_0 \tau}{2} (\mu - mu_0)^2 \bigg\} \\
    &\propto \tau^{(N+1)/2 + a_0 -1} \exp\{-b_0 \tau \} \, \exp\bigg\{-\frac{\tau}{2} \sum_{i=1}^N x_i^2 \bigg\} \, \exp\bigg\{-\frac{\lambda_0 \mu_0^2 \tau}{2} \bigg\} \, \exp\bigg\{-\frac{(\lambda_0 + N) \tau}{2} \big(\mu - \frac{\sum_{i=1}^N x_i + \lambda_0 \mu_0}{\lambda_0 + N} \big)^2 \bigg\} \\
    &\propto \tau^{(N+1)/2 + a_0 -1} \exp\bigg\{- \big(b_0 + \frac{1}{2} \sum_{i=1}^N x_i^2 + \frac{\lambda_0 \mu_0^2}{2} \big) \tau \bigg\} \, \exp\bigg\{- \frac{(\lambda_0 + N) \tau}{2} \big(\mu - \frac{\sum_{i=1}^N x_i + \lambda_0 \mu_0}{\lambda_0 + N} \big)^2 \bigg\}
\end{aligned}$$
于是可以发现，前半部分 $\tau^{(N+1)/2 + a_0 -1} \exp\bigg\{- \big(b_0 + \frac{1}{2} \sum_{i=1}^N x_i^2 + \frac{\lambda_0 \mu_0^2}{2} \big) \tau \bigg\}$ 是 Gamma 分布的形式，后半部分 $\exp\bigg\{- \frac{(\lambda_0 + N) \tau}{2} \big(\mu - \frac{\sum_{i=1}^N x_i + \lambda_0 \mu_0}{\lambda_0 + N} \big)^2 \bigg\}$ 是 Gaussian 分布的形式，所以后验分布的解析式可以直接写得：
$$
    p(\mu, \tau | X) = \mathcal N \bigg(\mu | \frac{\sum_{i=1}^N x_i + \lambda_0 \mu_0}{\lambda_0 + N}, \big[(\lambda_0 + N) \tau \big]^{-1} \bigg) \cdot \text{Ga} \bigg(\tau | a_0 + \frac{N}{2}, b_0 + \frac{1}{2} \sum_{i=1}^N x_i^2 + \frac{\lambda_0 \mu_0^2}{2} \bigg)
$$
对应参数的精确解为：
$$
    \mu_N = \frac{\lambda_0 \mu_0 + N \bar{x}}{\lambda_0 + N} \\
    \lambda_N = \lambda_0 + N \\
    a_N = a_0 + N / 2 \\
    b_N = b_0 + \frac{1}{2} \sum_{i=1}^N (x_i - \bar{x})^2 + \frac{\lambda_0 N (\bar{x} - \mu_0)^2}{2 (\lambda_0 + N)}
$$

那么假如不知道这个后验分布的解，比如一些非常复杂的后验分布无法求解，我们则需要通过变分推断(基于平均场)来近似这一后验。我们设该近似分布为：
$$
    q(\mu, \tau) = q_{\mu}(\mu) \, q_{\tau}(\tau)
$$
我们的目的是让 $q(\mu, \tau) \rightarrow p(\mu, \tau | X)$。

于是我们得到最优解 $q_{\mu}^{*}(\mu)$ 满足：
$$\begin{aligned}
    \log q_{\mu}^{*}(\mu) &= \mathbb E_{q_{\tau}(\tau)} [\log p(\mu, \tau, X)] \\
    &= \mathbb E_{q_{\tau}(\tau)} [\log p(X | \mu, \tau) + \log p(\mu | \tau)] + \text{const} \\
    &= \int_{\tau} q_{\tau}(\tau) \left[\frac{N}{2} \log (\tau) - \frac{\tau}{2} \sum_{i=1}^N (x_i - \mu)^2 - \frac{\lambda_0 \tau}{2} (\mu - \mu_0)^2 \right] + \text{const} \quad (将与 \mu 无关的项用 \text{const} 表示)\\
    &= -\frac{\mathbb E_{q_{\tau}(\tau)} [\tau]}{2} \left[\sum_{i=1}^N (x_i - \mu)^2 + \lambda_0 (\mu - \mu_0)^2 \right] + \text{const} \\
    &= - \frac{\mathbb E_{q_{\tau}(\tau)} [\tau] (N + \lambda_0)}{2} \left(\mu - \frac{N \bar{x} + \lambda_0 \mu_0}{N + \lambda_0} \right)^2 + \text{const}
\end{aligned}$$
所以，$q_{\mu}^*(\mu) = \mathcal N \big(\frac{N\bar{x} + \lambda_0 \mu_0}{N + \lambda_0}, \mathbb E_{q_{\tau}} [\tau] (N + \lambda_0) \big)$。
同理，$\log q_{\tau}^*(\tau) = \big(\underbrace{\frac{N}{2} + a_0}_{a_N} - 1 \big) \log(\tau) - \tau \big(\underbrace{b_0 + \frac{1}{2} \mathbb E_{q_{\mu}} [\sum_{i=1}^N (x_i - \mu)^2 + \lambda_0 (\mu - \mu_0)^2]}_{b_N} \big) + \text{const}$。
所以，$q_{\tau}^*(\tau) = \text{Gamma} (a_N, b_N)$

<br>

**可以得出结论**：
(1) 无须指定 $q_{\mu}(\mu)$ 和 $q_{\tau}(\tau)$ 的函数形式，因为它们可以从似然函数和共轭先验自动推导出来；

(2) 虽然我们假设了 $q_{\mu}(\mu)$ 和 $q_{\tau}(\tau)$ 相互独立，但求解结果表明它们是相互耦合的，即 $q_{\mu}(\mu)$ 依赖于 $q_{\tau}(\tau)$，而反过来 $q_{\tau}(\tau)$ 依赖于 $q_{\mu}(\mu)$。

(3) $\mu_N$ 和 $a_N$ 是固定常数，只有 $\lambda_N$ 和 $b_N$ 需要迭代更新。

<br>

### 2. Iterative optimization and Computing the expectation

根据上面的推断结果，进行一定顺序下的迭代优化求解：$\mathbb E [\tau] \longrightarrow q_{\mu}(\mu) \longrightarrow \mathbb E[\mu], \mathbb E[\mu^2] \longrightarrow q_{\tau}(\tau) \longrightarrow \mathbb E[\tau] \longrightarrow \dots$。
所以接下来的问题就是如何设置初始值 $\mathbb E[\tau]$，由于两个分布相互耦合，那么初始值一定会满足某些约束（为了简化计算，我们不妨令参数 $a_0 = b_0 = \mu_0 = \tau_0 = 0$
（即无信息先验）），为了实现更新，我们必须指定如何计算各种期望，接下来推导一下。
由于 $q_{\mu}(\mu) = \mathcal N(\mu | \mu_N, \lambda_N^{-1})$，我们得到：
$$
    \mathbb E_{q(\mu)}[\mu] = \mu_N = \bar{x} \\
    \mathbb E_{q(\mu)}[\mu^2] = \frac{1}{\lambda_N} + \mu_N^2 = frac{1}{N \mathbb E_{q(\tau)}[\tau]} + \bar{x}^2 \quad (\frac{a_N}{b_N} = \mathbb E_{q(\tau)}[\tau])
$$

由于 $q(\tau) = \text{Ga}(\tau | a_N, b_N)$，我们得到：
$$
    \mathbb E_{q(\tau)}[\tau] = \frac{a_N}{b_N} \\
    \frac{1}{\mathbb E_{q(\tau)}[\tau]} = \frac{b_N}{a_N} = \frac{1}{N} \sum_{i=1}^N (x_i - \bar{x})^2
$$
所以：$\mathbb E_{q(\tau)}[\tau] = \frac{N}{\sum_{i=1}^N (x_i - \bar{x})^2}$。我们由此确定了 $\mathbb E_{q(\tau)}[\tau]$ 的初值，接下来就可以进行迭代优化。

In [4]:
import numpy as np
import scipy